In [1]:
#initialise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.linear_model import LinearRegression
import datetime

In [53]:
#import jobs datasets

recruit = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Average Monthly Recruitment:Resignation Rates by Industry and Occupational Group, Quarterly/average-monthly-recruitment-resignation-rate-by-industry-and-occupational-group-level-3.csv',
                        header=0)

vacancy = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-by-industry-and-occupational-group-level3.csv',
                        header=0)

vacancyr = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-rate-by-industry-and-occupational-group-level3.csv',
                        header=0)

# join recruit, vacancy, vacancyr dataframes and filter for information and comms industry
jobs = vacancy.merge(vacancyr,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate']].merge(recruit,on=['quarter','industry1','industry2','industry3','occupation1'])[['quarter','industry1','industry2','industry3','occupation1','job_vacancy','job_vacancy_rate','recruitment_rate','resignation_rate']]

jobs = jobs[jobs['industry2']=='information and communications'].reset_index(drop=True)
jobs[['year', 'quarter']] = jobs['quarter'].str.split('-', 1, expand=True) # split the quarter column into year and quarter columns
jobs.insert(0, 'year', jobs.pop('year')) # move year column to the front
jobs['industry3'] = jobs['industry3'].replace(['telecommunications','broadcasting and publishing'], 'telecommunications, broadcasting and publishing') # unified industry3 values


# change the datatypes for all columns in jobs dataset and replace '-' with 0
jobs = jobs.replace('-',0)

convert_dict = {'year':str,
                'quarter':str,
                'industry1':str,
                'industry2':str,
                'industry3':str,
                'occupation1':str,
                'job_vacancy':int,
                'job_vacancy_rate':float,
                'recruitment_rate':float,
                'resignation_rate':float
                }

jobs = jobs.astype(convert_dict)

# drop the rows where numeric data is '0'
jobs = jobs[jobs['year'] != 0][jobs['job_vacancy'] != 0][jobs['job_vacancy_rate'] != 0][jobs['recruitment_rate'] != 0][jobs['resignation_rate'] != 0].reset_index(drop=True)

#view first 10 rows of dataset
jobs.head(10)

print(f"The jobs dataset is a combination of:\n'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Average Monthly Recruitment:Resignation Rates by Industry and Occupational Group, Quarterly/average-monthly-recruitment-resignation-rate-by-industry-and-occupational-group-level-3.csv',\n'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-by-industry-and-occupational-group-level3.csv', and\n'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-rate-by-industry-and-occupational-group-level3.csv'.")
print()
print(f"The datasets were merged and then filtered for jobs in the information and communication (ICT) industry. The year and quarter columns were created using string split from the original quarter column.")
print()
print(f"This is the shape of the dataset\n{jobs.shape}")
print()
print(f"This is the index of the dataset\n{jobs.index}")
print()
print(f"These are the columns in the dataset\n{jobs.columns}")
print()
print(f"The total number of non-NA values in this dataset is:\n{jobs.count()}")
print()
print(f"A summary of this dataset is shown below:")
print(jobs.info())
print()
print(f"A descriptive statistical summary of this dataset is shown below:\n{jobs[['job_vacancy','job_vacancy_rate','recruitment_rate','resignation_rate']].describe()}")

The jobs dataset is a combination of:
'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Average Monthly Recruitment:Resignation Rates by Industry and Occupational Group, Quarterly/average-monthly-recruitment-resignation-rate-by-industry-and-occupational-group-level-3.csv',
'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-by-industry-and-occupational-group-level3.csv', and
'/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Job Vacancy By Industry and Occupational Group, Quarterly/job-vacancy-rate-by-industry-and-occupational-group-level3.csv'.

The datasets were merged and then filtered for jobs in the information and communication (ICT) industry. The year and quarter columns were created using string split from the original quarter column.

This is the shape of the dataset

/var/folders/nv/c05ywcr140z_fcwc2kx0h69m0000gn/T/ipykernel_1904/2643841485.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  jobs[['year', 'quarter']] = jobs['quarter'].str.split('-', 1, expand=True) # split the quarter column into year and quarter columns
/var/folders/nv/c05ywcr140z_fcwc2kx0h69m0000gn/T/ipykernel_1904/2643841485.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  jobs = jobs[jobs['year'] != 0][jobs['job_vacancy'] != 0][jobs['job_vacancy_rate'] != 0][jobs['recruitment_rate'] != 0][jobs['resignation_rate'] != 0].reset_index(drop=True)


In [35]:
# import schools datasets

poly = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Polytechnics - Intake, Enrolment, Graduates/polytechnics-intake-enrolment-and-graduates-by-course.csv',
                    header=0)

uni = pd.read_csv('/Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Universities - Intake, Enrolment, Graduates/universities-intake-enrolment-and-graduates-by-course.csv',
                    header=0)

print(f"The polytechnic dataset was successfully loaded: /Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Polytechnics - Intake, Enrolment, Graduates/polytechnics-intake-enrolment-and-graduates-by-course.csv")
print()
print(f"This is the shape of the dataset\n{poly.shape}")
print()
print(f"This is the index of the dataset\n{poly.index}")
print()
print(f"These are the columns in the dataset\n{poly.columns}")
print()
print(f"The total number of non-NA values in this dataset is:\n{poly.count()}")
print()
print(f"A summary of this dataset is shown below:")
print(poly.info())
print()
print(f"A descriptive statistical summary of this dataset is shown below:\n{poly[['intake','enrolment','graduates']].describe()}")
print()
print()
print()
print()
print(f"The university dataset was successfully loaded: /Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Universities - Intake, Enrolment, Graduates/universities-intake-enrolment-and-graduates-by-course.csv")
print()
print(f"This is the shape of the dataset\n{uni.shape}")
print()
print(f"This is the index of the dataset\n{uni.index}")
print()
print(f"These are the columns in the dataset\n{uni.columns}")
print()
print(f"The total number of non-NA values in this dataset is:\n{uni.count()}")
print()
print(f"A summary of this dataset is shown below:")
print(uni.info())
print()
print(f"A descriptive statistical summary of this dataset is shown below:\n{uni[['intake','enrolment','graduates']].describe()}")

The polytechnic dataset was successfully loaded: /Users/iylia/Library/Mobile Documents/com~apple~CloudDocs/SP/Intro to Programming for DS/CA2/Data/Polytechnics - Intake, Enrolment, Graduates/polytechnics-intake-enrolment-and-graduates-by-course.csv

This is the shape of the dataset
(408, 6)

This is the index of the dataset
RangeIndex(start=0, stop=408, step=1)

These are the columns in the dataset
Index(['year', 'sex', 'course', 'intake', 'enrolment', 'graduates'], dtype='object')

The total number of non-NA values in this dataset is:
year         408
sex          408
course       408
intake       408
enrolment    408
graduates    408
dtype: int64

A summary of this dataset is shown below:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       408 non-null    int64 
 1   sex        408 non-null    object
 2   course     408 non-null    object
 

In [48]:
poly['course'].unique()

# filter poly data for ICT industry degrees
poly_ict = poly[poly['course'].isin(['Engineering Sciences','Information Technology','Mass Communication'])].reset_index(drop=True)

uni['course'].unique()

# filter uni data for ICT industry degrees
uni_ict = uni[uni['course'].isin(['Engineering Sciences','Information Technology','Mass Communication'])].reset_index(drop=True)

# merge poly and uni datasets
poly_uni_ict = poly_ict.merge(uni_ict, on=['year','sex','course'], suffixes=('_poly','_uni'))

print(poly_uni_ict.head(10))

   year sex                  course  intake_poly  enrolment_poly  \
0  2005  MF    Engineering Sciences         7826           22462   
1  2005   F    Engineering Sciences         2097            5939   
2  2005  MF  Information Technology         4122           11607   
3  2005   F  Information Technology         1887            5065   
4  2005  MF      Mass Communication          448            1426   
5  2005   F      Mass Communication          324            1029   
6  2006  MF    Engineering Sciences         8209           23713   
7  2006   F    Engineering Sciences         2108            6197   
8  2006  MF  Information Technology         3972           11753   
9  2006   F  Information Technology         1728            5134   

   graduates_poly  intake_uni  enrolment_uni  graduates_uni  
0            6536        4028          16599           3859  
1            2005        1170           4868           1200  
2            3356         773           2356            562  
3  

In [59]:
jobs.head(10)

jobs_mean = jobs[jobs['occupation1'] == 'total'][['year','job_vacancy','job_vacancy_rate','recruitment_rate','resignation_rate']].groupby('year').mean().round(2)

